In [ ]:
# mount Google Drive so you can access the results
from google.colab import drive
drive.mount('/content/drive')

In [284]:
import pandas as pd
import gspread
from oauth2client.client import GoogleCredentials
from google.colab import auth
import json
auth.authenticate_user()

In [364]:
gc = gspread.authorize(GoogleCredentials.get_application_default())
wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1-XISAVZLTrBa21v7w0C0ygbUqL3vnDwtGFfTrJ2owtA')
sheet = wb.worksheet('Sheet1')

In [ ]:
orig = pd.DataFrame.from_records(sheet.get_all_values())
results = orig.drop(orig.columns[-2:], axis=1)
results.head()

In [367]:
cifar10_res = results.drop(results.columns[5:], axis=1)
cifar10_res = cifar10_res.rename(columns=cifar10_res.iloc[1]).drop(cifar10_res.index[0:2])
cifar100_res = results.drop(results.columns[1:5], axis=1)
cifar100_res = cifar100_res.rename(columns=cifar100_res.iloc[1]).drop(cifar100_res.index[0:2])

In [368]:
cifar10_res['Dataset'] = 'cifar10'
cifar100_res['Dataset'] = 'cifar100'

In [369]:
results = cifar10_res.append(cifar100_res)
stats_cols = results.columns[1:5]
results[stats_cols] = results[stats_cols].apply(pd.to_numeric, errors='coerce')

In [ ]:
results.head()

In [371]:
grouped = results.groupby(['Model', 'Dataset'])[numeric_cols].last().to_dict(orient='index')

In [372]:
output = {'trained': None, 'soa': None}
for (model, dataset) in grouped.keys():
  acc1, acc5, ttest1, ttest5 = grouped[(model, dataset)].values()
  stats_dict= {'accuracy': [acc1, acc5], 't_test': [ttest1, ttest5]}
  if output['trained'] is None: output['trained'] = {model: {dataset: stats_dict}}
  elif model not in output['trained'].keys(): output['trained'][model] = {dataset: stats_dict}
  else: output['trained'][model][dataset] = stats_dict

In [373]:
details = orig.drop(orig.columns[1:-2], axis=1)
details = details.rename(columns=details.iloc[1]).drop(details.index[0:2])
details.columns = [col.lower() for col in details.columns]

In [ ]:
details_dict = details.groupby('model')['details', 'reason'].last().to_dict(orient='index')

In [375]:
for model in output['trained'].keys():
  output['trained'][model].update(details_dict[model])

In [ ]:
print(json.dumps(output, indent=2))

In [384]:
json.dump(output, open('/content/drive/MyDrive/SWENG 2/results.json', 'w'), indent=2)